In [1]:

import numpy as np
import pandas as pd
from tqdm import tqdm
import gc
import os
from glob import glob
from multiprocessing import Pool
nthread = 16
from time import time
#from collections import defaultdict
import utils

In [2]:
trte = utils.read_pickles('../data/train', ['ip', 'app', 'device', 'os', 'channel', 'click_time'])

gc.collect()


100%|██████████| 10/10 [00:18<00:00,  1.80s/it]


29

In [5]:
count_keys = ('ip',)

In [12]:
st = time()


gc.collect()
print(count_keys)
count_keys = list(count_keys)

count_keys_ = '-'.join(count_keys)
keys = count_keys+['click_time']

df = trte.sort_values(keys)
df['prekey_match']  = ( df[count_keys]==df[count_keys].shift() ).all(1)*1
df['nextkey_match'] = ( df[count_keys]==df[count_keys].shift(-1) ).all(1)*1
gc.collect()

print(f'done {time()-st}')

['ip']
done 237.24336671829224


In [13]:
df

,ip,app,device,os,channel,click_time,prekey_match,nextkey_match
124211656,1,2,1,15,477,2017-11-08 17:25:31,0,1
124213049,1,12,1,15,265,2017-11-08 17:25:35,1,1
124213379,1,64,1,15,459,2017-11-08 17:25:36,1,1
124213380,1,6,1,15,459,2017-11-08 17:25:36,1,1
124679987,1,2,1,2,477,2017-11-08 17:47:39,1,1
126054437,1,13,1,28,469,2017-11-08 19:37:16,1,1
126241315,1,2,1,6,477,2017-11-08 19:59:36,1,1
126290289,1,2,1,49,477,2017-11-08 20:05:42,1,1
126312169,1,2,1,2,477,2017-11-08 20:08:39,1,1
126325989,1,2,1,2,477,2017-11-08 20:10:32,1,1


In [14]:
st = time()

col = []
c = 'preApp_'+count_keys_
col.append(c)
df[c] = df.app.shift()
df.loc[df.prekey_match==0, c] = -1

c = 'nextApp_'+count_keys_
col.append(c)
df[c] = df.app.shift(-1)
df.loc[df.nextkey_match==0, c] = -1
gc.collect()

print(f'done {time()-st}')

done 26.455004930496216


In [15]:
df

,ip,app,device,os,channel,click_time,prekey_match,nextkey_match,preApp_ip,nextApp_ip
124211656,1,2,1,15,477,2017-11-08 17:25:31,0,1,-1.0,12.0
124213049,1,12,1,15,265,2017-11-08 17:25:35,1,1,2.0,64.0
124213379,1,64,1,15,459,2017-11-08 17:25:36,1,1,12.0,6.0
124213380,1,6,1,15,459,2017-11-08 17:25:36,1,1,64.0,2.0
124679987,1,2,1,2,477,2017-11-08 17:47:39,1,1,6.0,13.0
126054437,1,13,1,28,469,2017-11-08 19:37:16,1,1,2.0,2.0
126241315,1,2,1,6,477,2017-11-08 19:59:36,1,1,13.0,2.0
126290289,1,2,1,49,477,2017-11-08 20:05:42,1,1,2.0,2.0
126312169,1,2,1,2,477,2017-11-08 20:08:39,1,1,2.0,2.0
126325989,1,2,1,2,477,2017-11-08 20:10:32,1,1,2.0,13.0
